In [3]:
from bs4 import BeautifulSoup, NavigableString, Tag
import requests
import pandas as pd
import time
from PIL import Image
from pytesseract import pytesseract
from io import BytesIO
import base64
import pytesseract

In [4]:
pytesseract.pytesseract.tesseract_cmd = 'C:/Program Files/Tesseract-OCR/tesseract.exe'  # your path may be different

In [5]:
global data

In [6]:
#get text from image
def getFeatureTextFromImg(imgUrl):
    url = imgUrl
    headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'}
    r = requests.get(url, headers=headers)
    img = Image.open(BytesIO(r.content))
    text = pytesseract.image_to_string(img)
    return text

In [8]:
def getDetailsData(detailsUrl):
    #print(detailsUrl)
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0'}
    detialRequest = requests.get(detailsUrl,headers=headers)
    detailContent = detialRequest.content
    detailSoup = BeautifulSoup(detailContent,features="html.parser")
    details_container=detailSoup.find('aside',{'class':'left-side'})
    #location=details_container.find('div',{'class','location'})
    return details_container


In [112]:
global df
df=pd.DataFrame()

In [116]:
def getCarsData(page_from,page_to):
    global df
    data=[]
    headers = {'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:79.0) Gecko/20100101 Firefox/79.0'}
    for pageNo in range(page_from,page_to):
        print(pageNo,df.shape[0])
        url='https://ksa.carswitch.com/en/saudi/used-cars/search?page={}'.format(pageNo)
        r = requests.get(url,headers=headers)
        content = r.content
        soup = BeautifulSoup(content,features="html.parser")
        
        #soup = BeautifulSoup(data,'html.parser')
        #print(soup)
        #get div with id main-listing-div
        soup = soup.find('section',{'id':'main-listing-div'})
        #print(soup)
        features={}
        items=soup.findAll('div',{'class':'show-price-badge'})
        #print(items)
        for index,item in enumerate(items):
            features={}
            title=item.find('div',{'class':'title'}).find('a').text
            price=item.find('div',{'class':'title'}).find('span').text
            model_year=item.find('div',{'class':'info'}).find('div',{'class','car-mileage'}).text.split(' · ')[0]
            odo=item.find('div',{'class':'info'}).find('div',{'class','car-mileage'}).text.split(' · ')[1]
            #get more data from details page
            detailsUrl='https://ksa.carswitch.com'+item.find('figure',{'class','image-holder'}).find('a',href=True)['href']
            print(detailsUrl)
            details_html=getDetailsData(detailsUrl)
            location=details_html.find('div',{'class','location'}).find('span').text
            aminities_html=details_html.findAll('div',{'class','car-overview__features'})
            car_features_html=details_html.find('div',{'class','car-overview__features-detail'}).findAll('div',{'class','feature-list__item'})
            #print(features_html)
            #print(features_html.length)
            features={}
            aminities=[]
            #print(features_html[:-1])
            for category in aminities_html[:-1]:
                for index,feature in enumerate(category.findAll('div',class_='item')):
                    if len(feature["class"]) != 1:#if the div has more than one class
                        continue
                    else:
                        img=feature.find('img')
                        if img.has_attr('data-src'):
                            imgUrl="https://ksa.carswitch.com"+img['data-src']
                        else:
                            imgUrl="https://ksa.carswitch.com"+img['src']
                        imgText=getFeatureTextFromImg(imgUrl)
                        if imgText == '' and feature.has_attr('data-searchable'):
                            aminities.append(feature['data-searchable'] )
                        else:
                            aminities.append(imgText.replace("\n"," ").strip().replace("\x0c", "") )
                        #print(imgText)
           
            try:
                features['title']=title
                features['price']=price
                features['model_year']=model_year
                features['odo']=odo
                features['aminities']=aminities
                 # car features details
                for feature in car_features_html:
                    #print(car_features_html)
                    #print(feature)
                    name=feature.find('div',{'class':'feature-name'}).find(text=True).strip().replace(' ','_').lower()
                    value=feature.find('div',{'class':'feature-value'}).find(text=True).strip()
                    print(name,value)
                    features[name]=value
                print(features)
                df=df.append(features,ignore_index=True)
                #print(index,title,'   |',price,'  |',model_year,'  |',odo,' |',location," |",aminities)
                #break
            except Exception as ex :
                print(ex)
                continue

In [117]:
t0 = time.time()
getCarsData(1,2)
t1 = time.time()
print('\n\nTime comsumed: ', t1-t0)

1 1
https://ksa.carswitch.com/en/riyadh/used-car/mercedes/c200/2015/320111-323943
specs GCC /Saudi Specs
number_of_seats 5
roof_type Panoramic Roof
drive_type 2WD
number_of_cylinders 4
body_type Sedan
transmission_type Automatic
fuel_type Petrol
rim_size R18
first_owner Yes
accidents_(not_verified) 2 accidents
currently_financed? No
seller_nationality Saudi Arabia
no._of_keys 1
{'title': 'Mercedes C200 I4 2.0L AMG', 'price': '\nSAR 81,500 ', 'model_year': '2015 ', 'odo': '109,000 KM', 'aminities': ['fal Parking sensors', 'Cruise Control', '@  Navigation', 'W  Reversing Camera', '{Or Automated Parking', 'A-©  Digital display', 'Steering wheel control', 'Paddle shifters', '>  Bluetooth', '{0} premium sound system', 'ic  Leather Seats', 'vi.  Ambient lighting', '\\s  Power seats', '%  Lumbar support'], 'specs': 'GCC /Saudi Specs', 'number_of_seats': '5', 'roof_type': 'Panoramic Roof', 'drive_type': '2WD', 'number_of_cylinders': '4', 'body_type': 'Sedan', 'transmission_type': 'Automatic', 

C:\Users\saad\AppData\Local\Temp\ipykernel_16336\3148207880.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(features,ignore_index=True)


specs GCC /Saudi Specs
service_history Yes in Garage
number_of_seats 5
roof_type Sun Roof
drive_type 2WD
number_of_cylinders 4
color Brown
body_type SUV
transmission_type Automatic
fuel_type Petrol
rim_size 19
first_owner Yes
accidents_(not_verified) No
currently_financed? No
seller_nationality Jordan
no._of_keys 2
{'title': 'Mazda CX-5 I4 2.5L', 'price': '\nSAR 65,000 ', 'model_year': '2017 ', 'odo': '62,000 KM', 'aminities': ['fal Parking sensors', 'Cruise Control', '@  Navigation', 'W  Reversing Camera', 'Steering wheel control', '>  Bluetooth', '{0} premium sound system', 'ic  Leather Seats', 'Push Start', '\\s  Power seats', '(sie)  LED headlights', 'OF  Fog lights', 'Sy  Brake assist'], 'specs': 'GCC /Saudi Specs', 'service_history': 'Yes in Garage', 'number_of_seats': '5', 'roof_type': 'Sun Roof', 'drive_type': '2WD', 'number_of_cylinders': '4', 'color': 'Brown', 'body_type': 'SUV', 'transmission_type': 'Automatic', 'fuel_type': 'Petrol', 'rim_size': '19', 'first_owner': 'Yes', 

C:\Users\saad\AppData\Local\Temp\ipykernel_16336\3148207880.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(features,ignore_index=True)


specs GCC /Saudi Specs
service_history Yes In Agency
number_of_seats 7
drive_type 4WD
number_of_cylinders 4
color White
body_type SUV
transmission_type Automatic
fuel_type Petrol
rim_size 17
first_owner Yes
accidents_(not_verified) 1 accident
currently_financed? No
seller_nationality Jordan
no._of_keys 2
{'title': 'Toyota Prado I4 2.7L TXL', 'price': '\nSAR 100,000 ', 'model_year': '2016 ', 'odo': '86,000 KM', 'aminities': ['fal Parking sensors', 'Cruise Control', 'W  Reversing Camera', 'Steering wheel control', '>  Bluetooth', '{0} premium sound system', '(sie)  LED headlights', 'OF  Fog lights', 'Sy  Brake assist'], 'specs': 'GCC /Saudi Specs', 'service_history': 'Yes In Agency', 'number_of_seats': '7', 'drive_type': '4WD', 'number_of_cylinders': '4', 'color': 'White', 'body_type': 'SUV', 'transmission_type': 'Automatic', 'fuel_type': 'Petrol', 'rim_size': '17', 'first_owner': 'Yes', 'accidents_(not_verified)': '1 accident', 'currently_financed?': 'No', 'seller_nationality': 'Jordan'

...

specs GCC /Saudi Specs
service_history Yes In Agency
number_of_seats 7
roof_type Panoramic Roof
drive_type 2WD
number_of_cylinders 4
body_type SUV
transmission_type Automatic
fuel_type Petrol
rim_size 20
first_owner Yes
accidents_(not_verified) No
currently_financed? No
seller_nationality Egypt
no._of_keys 2
{'title': 'Jetour X70 I4 1.5L', 'price': '\nSAR 61,500 ', 'model_year': '2022 ', 'odo': '800 KM', 'aminities': ['A  Headsup display', 'fal Parking sensors', 'Cruise Control', '@  Navigation', 'W  Reversing Camera', '8  Birds eye view camera', 'Gy C__)  1  Rear mirror camera', '{Or Automated Parking', '(S)  Adaptive cruise control', 'A-©  Digital display', 'Steering wheel control', 'Paddle shifters', 'oP  Auto dimming mirror', '>  Bluetooth', 'o  Apple carplay/ Android', '{0} premium sound system', 'ic  Leather Seats', 'Push Start', 'Remote start', 'vi.  Ambient lighting', 'Keyless entry', 'IS  Memory seats', 'i  ey  Power folding mirrors', '18  Heated-cooled seats', '%  Lumbar supp

C:\Users\saad\AppData\Local\Temp\ipykernel_16336\3148207880.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(features,ignore_index=True)


specs GCC /Saudi Specs
number_of_seats 5
drive_type AWD
number_of_cylinders 4
color Blue
body_type SUV
transmission_type Automatic
fuel_type Petrol
rim_size 17
accidents_(not_verified) No
currently_financed? No
no._of_keys 1
{'title': 'Jeep Compass I4 2.4L', 'price': '\nSAR 36,000 ', 'model_year': '2015 ', 'odo': '105,000 KM', 'aminities': ['Cruise Control', 'A-©  Digital display', 'Steering wheel control'], 'specs': 'GCC /Saudi Specs', 'number_of_seats': '5', 'drive_type': 'AWD', 'number_of_cylinders': '4', 'color': 'Blue', 'body_type': 'SUV', 'transmission_type': 'Automatic', 'fuel_type': 'Petrol', 'rim_size': '17', 'accidents_(not_verified)': 'No', 'currently_financed?': 'No', 'no._of_keys': '1'}
https://ksa.carswitch.com/en/riyadh/used-car/mitsubishi/pajero/2017/288177-292008


C:\Users\saad\AppData\Local\Temp\ipykernel_16336\3148207880.py:70: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df=df.append(features,ignore_index=True)


PermissionError: [WinError 32] The process cannot access the file because it is being used by another process: 'C:\\Users\\saad\\AppData\\Local\\Temp\\tess_dmr56v37.PNG'

In [118]:
df.head()

,title,price,model_year,odo,aminities,specs,number_of_seats,roof_type,drive_type,number_of_cylinders,...,transmission_type,fuel_type,rim_size,first_owner,accidents_(not_verified),currently_financed?,seller_nationality,no._of_keys,color,service_history
0,Mercedes C200 I4 2.0L AMG,"\nSAR 81,500",2015,"109,000 KM","[fal Parking sensors, Cruise Control, @ Navig...",GCC /Saudi Specs,5,Panoramic Roof,2WD,4,...,Automatic,Petrol,R18,Yes,2 accidents,No,Saudi Arabia,1,NaN,NaN
1,Mercedes C200 I4 2.0L AMG,"\nSAR 81,500",2015,"109,000 KM","[fal Parking sensors, Cruise Control, @ Navig...",GCC /Saudi Specs,5,Panoramic Roof,2WD,4,...,Automatic,Petrol,R18,Yes,2 accidents,No,Saudi Arabia,1,NaN,NaN
2,Mazda CX-5 I4 2.5L,"\nSAR 65,000",2017,"62,000 KM","[fal Parking sensors, Cruise Control, @ Navig...",GCC /Saudi Specs,5,Sun Roof,2WD,4,...,Automatic,Petrol,19,Yes,No,No,Jordan,2,Brown,Yes in Garage
3,Toyota Prado I4 2.7L TXL,"\nSAR 100,000",2016,"86,000 KM","[fal Parking sensors, Cruise Control, W Rever...",GCC /Saudi Specs,7,NaN,4WD,4,...,Automatic,Petrol,17,Yes,1 accident,No,Jordan,2,White,Yes In Agency
4,Jetour X70 I4 1.5L,"\nSAR 61,500",2022,800 KM,"[A Headsup display, fal Parking sensors, Crui...",GCC /Saudi Specs,7,Panoramic Roof,2WD,4,...,Automatic,Petrol,20,Yes,No,No,Egypt,2,NaN,Yes In Agency


In [76]:
'2017 · 210,000 km'.split(' · ')[0]

'2017'